<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
# Parameters
test_start = "2026-01-10 00:00"


# Credit Card Fraud Detection - Neural Network Model

This notebook trains a feedforward neural network for credit card fraud detection.

**Key differences from XGBoost notebook:**
- Uses PyTorch neural network instead of XGBoost
- Creates Feature View v2 with MinMaxScaler and OneHotEncoder transformations
- 3-layer feedforward network with ~30k neurons
- L2 regularization (weight_decay) + Dropout layers
- Class weights to handle severe class imbalance (191:1)

In [2]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

root_dir = Path().absolute()
# Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
if root_dir.parts[-1:] == ('ccfraud',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
root_dir = str(root_dir) 

print(f"Root dir: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Root dir: /home/jdowling/Projects/mlfs-book
HopsworksSettings initialized!


In [3]:
import hopsworks
import pandas as pd
import numpy as np
import shutil
import os

proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()

2026-01-17 09:28:23,342 INFO: Initializing external client
2026-01-17 09:28:23,343 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-17 09:28:24,659 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/120


## Get Feature Groups

In [4]:
merchant_fg = fs.get_feature_group("merchant_details", version=1)
account_fg = fs.get_feature_group("account_details", version=1)
bank_fg = fs.get_feature_group("bank_details", version=1)
card_fg = fs.get_feature_group("card_details", version=1)
cc_trans_aggs_fg = fs.get_feature_group("cc_trans_aggs_fg", version=1)
cc_trans_fg = fs.get_feature_group("cc_trans_fg", version=1)

## Build Feature Query

In [5]:
subtree1 = cc_trans_aggs_fg.select_except(['t_id','cc_num','account_id','bank_id','event_time'])\
    .join(account_fg.select(['debt_end_prev_month']), on="account_id", join_type="inner")\
    .join(bank_fg.select(['credit_rating', 'days_since_bank_cr_changed', 'country']), on="bank_id", join_type="inner")

In [6]:
selection = cc_trans_fg.select_except(['t_id', 'cc_num', 'merchant_id', 'account_id', 'ip_address', 'ts'])\
    .join(merchant_fg.select_features(), prefix="merchant_", on="merchant_id", join_type="inner")\
    .join(subtree1, on="cc_num")

2026-01-17 09:28:29,977 INFO: Using ['category', 'country', 'cnt_chrgeback_prev_day', 'cnt_chrgeback_prev_week', 'cnt_chrgeback_prev_month'] from feature group `merchant_details` as features for the query. To include primary key and event time use `select_all`.


## Create Feature View with Transformations

Create a new feature view (version 2) with:
- MinMaxScaler on `amount` feature
- OneHotEncoder on categorical features: `merchant_category`, `merchant_country`, `country`

In [7]:
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
one_hot_encoder = fs.get_transformation_function(name="one_hot_encoder")

# Define transformation functions for neural network preprocessing
transformation_functions = [
    min_max_scaler("amount"),
    one_hot_encoder("merchant_category"),
    one_hot_encoder("merchant_country"),
    one_hot_encoder("country"),
]

In [8]:
# Create Feature View version 2 with transformations
fv = fs.get_or_create_feature_view(
    name="cc_fraud_fv_nn", 
    version=1, 
    description="Features for credit card fraud NN model with MinMaxScaler and OneHotEncoder",
    query=selection,
    labels=['is_fraud'],
    inference_helper_columns=['prev_card_present', 'prev_ip_transaction', 'prev_ts_transaction'],
    transformation_functions=transformation_functions
)

print(f"Feature View: {fv.name}, version: {fv.version}")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [11]:
fv = fs.get_feature_view(
    name="cc_fraud_fv_nn", 
    version=1
)

# fv.delete()

## Train/Test Split

In [12]:
# Parameters (injected by papermill)
test_start = "2026-01-09 00:00"

In [13]:
X_train, X_test, y_train, y_test = fv.train_test_split(test_start=test_start)

print(f"Training data: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
print(f"Test data: {X_test.shape[0]:,} samples, {X_test.shape[1]} features")
X_train.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.80s) 
2026-01-17 09:58:45,447 INFO: Computing insert statistics
2026-01-17 09:58:48,513 INFO: Computing insert statistics
2026-01-17 09:58:48,875 INFO: Computing insert statistics
Training data: 339,085 samples, 77 features
Test data: 98,259 samples, 77 features


,time_since_last_trans,days_to_card_expiry,haversine_distance,merchant_cnt_chrgeback_prev_day,merchant_cnt_chrgeback_prev_week,merchant_cnt_chrgeback_prev_month,num_trans_last_10_mins,sum_trans_last_10_mins,num_trans_last_hour,sum_trans_last_hour,...,one_hot_encoder_merchant_country_10,one_hot_encoder_merchant_country_11,one_hot_encoder_merchant_country_12,one_hot_encoder_merchant_country_13,one_hot_encoder_merchant_country_14,one_hot_encoder_merchant_country_15,one_hot_encoder_merchant_country_16,one_hot_encoder_merchant_country_17,one_hot_encoder_merchant_country_18,one_hot_encoder_merchant_country_19
0,-2588489,0,True,0.75,5.82,20.56,1,13.98,1,13.98,...,False,False,False,False,False,False,False,False,False,False
1,-2577288,0,True,0.91,5.46,30.62,1,47.46,1,47.46,...,False,False,False,False,False,False,False,False,False,False
2,-2584271,0,True,4.13,26.89,115.14,1,66.46,1,66.46,...,False,False,False,False,False,False,False,False,False,False
3,-2588248,0,True,0.93,6.57,29.12,1,13.71,1,13.71,...,True,False,False,False,False,False,False,False,False,False
4,-2584019,0,True,8.43,54.55,297.70,1,35.36,1,35.36,...,False,True,False,False,False,False,False,False,False,False


## Class Imbalance Analysis

In [14]:
n_negative = (y_train["is_fraud"] == False).sum()
n_positive = (y_train["is_fraud"] == True).sum()
class_weight_ratio = n_negative / n_positive

print("=" * 80)
print("CLASS IMBALANCE ANALYSIS")
print("=" * 80)
print(f"Negative samples (non-fraud): {n_negative:,}")
print(f"Positive samples (fraud):     {n_positive:,}")
print(f"Imbalance ratio:              {class_weight_ratio:.2f}:1")
print(f"\nThis will be used as pos_weight in BCEWithLogitsLoss")
print(f"to give ~{class_weight_ratio:.0f}x more weight to fraud cases during training.")

CLASS IMBALANCE ANALYSIS
Negative samples (non-fraud): 337,246
Positive samples (fraud):     1,839
Imbalance ratio:              183.39:1

This will be used as pos_weight in BCEWithLogitsLoss
to give ~183x more weight to fraud cases during training.


## Prepare Data for PyTorch

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.impute import SimpleImputer

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Handle missing values with median imputation
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_imputed).to(device)
y_train_tensor = torch.FloatTensor(y_train.values.ravel()).to(device)
X_test_tensor = torch.FloatTensor(X_test_imputed).to(device)
y_test_tensor = torch.FloatTensor(y_test.values.ravel()).to(device)

# Create DataLoaders
batch_size = 256
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train_tensor.shape[1]
print(f"Input dimension: {input_dim}")
print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

Using device: cpu
Input dimension: 77
Training batches: 1325
Test batches: 384


## Neural Network Architecture

3-layer feedforward network with ~30k neurons:
- Layer 1: 15,000 neurons + ReLU + Dropout
- Layer 2: 10,000 neurons + ReLU + Dropout
- Layer 3: 5,000 neurons + ReLU + Dropout
- Output: 1 neuron (binary classification)

In [17]:
class FraudDetectorNN(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.3):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 15000),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(15000, 10000),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(5000, 1)
        )
    
    def forward(self, x):
        return self.network(x)

# Initialize model
model = FraudDetectorNN(input_dim, dropout_rate=0.3).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("=" * 80)
print("NEURAL NETWORK ARCHITECTURE")
print("=" * 80)
print(model)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"\nNeuron count: 15,000 + 10,000 + 5,000 = 30,000")

NEURAL NETWORK ARCHITECTURE
FraudDetectorNN(
  (network): Sequential(
    (0): Linear(in_features=77, out_features=15000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=15000, out_features=10000, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=10000, out_features=5000, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=5000, out_features=1, bias=True)
  )
)

Total parameters: 201,190,001
Trainable parameters: 201,190,001

Neuron count: 15,000 + 10,000 + 5,000 = 30,000


## Training Configuration

- Loss: BCEWithLogitsLoss with pos_weight for class imbalance
- Optimizer: Adam with weight_decay=0.01 (L2 regularization)
- Learning rate: 0.001 with ReduceLROnPlateau scheduler
- Early stopping: patience=10

In [19]:
# Loss function with class weight for imbalanced data
pos_weight = torch.tensor([class_weight_ratio]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# Optimizer with L2 regularization (weight_decay)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

print("=" * 80)
print("TRAINING CONFIGURATION")
print("=" * 80)
print(f"Loss function: BCEWithLogitsLoss")
print(f"  - pos_weight: {class_weight_ratio:.2f} (class imbalance handling)")
print(f"Optimizer: Adam")
print(f"  - learning_rate: 0.001")
print(f"  - weight_decay: 0.01 (L2 regularization)")
print(f"Scheduler: ReduceLROnPlateau")
print(f"  - patience: 5, factor: 0.5")
print(f"Early stopping: patience=10")
print(f"Batch size: {batch_size}")
print(f"Max epochs: 100")

TRAINING CONFIGURATION
Loss function: BCEWithLogitsLoss
  - pos_weight: 183.39 (class imbalance handling)
Optimizer: Adam
  - learning_rate: 0.001
  - weight_decay: 0.01 (L2 regularization)
Scheduler: ReduceLROnPlateau
  - patience: 5, factor: 0.5
Early stopping: patience=10
Batch size: 256
Max epochs: 100


## Train the Model

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

def evaluate_model(model, data_loader, criterion, device):
    """Evaluate model and return loss and PR-AUC."""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
            
            probs = torch.sigmoid(outputs).cpu().numpy()
            all_preds.extend(probs)
            all_labels.extend(y_batch.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    
    # Calculate PR-AUC
    precision, recall, _ = precision_recall_curve(all_labels, all_preds)
    pr_auc = auc(recall, precision)
    
    return avg_loss, pr_auc

# Training loop with early stopping
num_epochs = 100
early_stopping_patience = 10
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None
history = {'train_loss': [], 'val_loss': [], 'val_pr_auc': []}

print("=" * 80)
print("TRAINING NEURAL NETWORK")
print("=" * 80)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0
    
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    # Validation phase
    val_loss, val_pr_auc = evaluate_model(model, test_loader, criterion, device)
    
    # Update learning rate scheduler
    scheduler.step(val_loss)
    
    # Store history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_pr_auc'].append(val_pr_auc)
    
    # Print progress
    if (epoch + 1) % 5 == 0 or epoch == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1:3d}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | "
              f"Val PR-AUC: {val_pr_auc:.4f} | "
              f"LR: {current_lr:.6f}")
    
    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print(f"\nEarly stopping triggered at epoch {epoch+1}")
            break

# Restore best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\nRestored best model with validation loss: {best_val_loss:.4f}")

print("\nTraining complete!")

TRAINING NEURAL NETWORK


## Training History Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss', color='blue')
axes[0].plot(history['val_loss'], label='Val Loss', color='orange')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# PR-AUC plot
axes[1].plot(history['val_pr_auc'], label='Val PR-AUC', color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('PR-AUC')
axes[1].set_title('Validation PR-AUC')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
training_history_fig = fig
plt.show()

## Model Evaluation

In [ ]:
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_recall_curve, auc,
    precision_score, recall_score, f1_score
)

# Get predictions
model.eval()
with torch.no_grad():
    y_pred_probs = torch.sigmoid(model(X_test_tensor)).cpu().numpy().squeeze()
    y_pred = (y_pred_probs >= 0.5).astype(int)

y_test_np = y_test.values.ravel()

# Confusion Matrix
cm = confusion_matrix(y_test_np, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
df_cm = pd.DataFrame(cm, 
                     index=['True Non-Fraud', 'True Fraud'],
                     columns=['Pred Non-Fraud', 'Pred Fraud'])

sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar_kws={'label': 'Count'})
ax.set_title('Confusion Matrix - Neural Network Fraud Detection', fontsize=14, fontweight='bold')
ax.set_ylabel('Actual', fontsize=12)
ax.set_xlabel('Predicted', fontsize=12)

plt.tight_layout()
cm_fig = fig
plt.show()

print("=" * 80)
print("CONFUSION MATRIX BREAKDOWN")
print("=" * 80)
print(f"True Negatives:  {cm[0,0]:5,} (correctly identified non-fraud)")
print(f"False Positives: {cm[0,1]:5,} (non-fraud flagged as fraud)")
print(f"False Negatives: {cm[1,0]:5,} (fraud missed - CRITICAL)")
print(f"True Positives:  {cm[1,1]:5,} (correctly identified fraud)")

In [ ]:
# Classification Metrics
print("=" * 80)
print("CLASSIFICATION REPORT")
print("=" * 80)
report_dict = classification_report(y_test_np, y_pred, 
                                   target_names=['Non-Fraud', 'Fraud'],
                                   output_dict=True)
print(classification_report(y_test_np, y_pred, target_names=['Non-Fraud', 'Fraud']))

# Calculate key metrics
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
f1 = f1_score(y_test_np, y_pred)

# PR-AUC
precision_curve, recall_curve, _ = precision_recall_curve(y_test_np, y_pred_probs)
pr_auc = auc(recall_curve, precision_curve)

print("=" * 80)
print("KEY METRICS SUMMARY")
print("=" * 80)
print(f"PR-AUC Score:         {pr_auc:.4f}  <- More important for imbalanced data")
print(f"Precision (Fraud):    {precision:.4f}")
print(f"Recall (Fraud):       {recall:.4f}")
print(f"F1-Score (Fraud):     {f1:.4f}")

# Store metrics for model registry
metrics_dict = {
    'pr_auc': pr_auc,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'accuracy': report_dict['accuracy']
}

print("\nInterpretation:")
print(f"  - Precision: {precision*100:.1f}% of predicted frauds are actually fraudulent")
print(f"  - Recall: {recall*100:.1f}% of actual frauds were detected")
print(f"  - PR-AUC: {pr_auc:.4f} measures precision-recall tradeoff (higher is better)")

## Precision-Recall Curve

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(recall_curve, precision_curve, color='blue', lw=2, label=f'PR Curve (AUC = {pr_auc:.4f})')
ax.fill_between(recall_curve, precision_curve, alpha=0.2, color='blue')
ax.set_xlabel('Recall', fontsize=12)
ax.set_ylabel('Precision', fontsize=12)
ax.set_title('Precision-Recall Curve - Neural Network', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])

plt.tight_layout()
pr_curve_fig = fig
plt.show()

## Save Model Artifacts

In [ ]:
model_dir = "cc_fraud_nn_model"
images_dir = model_dir + "/images"
os.makedirs(images_dir, exist_ok=True)

print(f"Model artifacts will be saved to: {model_dir}")

In [ ]:
import joblib

print("=" * 80)
print("SAVING MODEL ARTIFACTS")
print("=" * 80)

# Save PyTorch model
model_path = model_dir + "/cc_fraud_nn_model.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'input_dim': input_dim,
    'dropout_rate': 0.3,
    'class_weight_ratio': class_weight_ratio,
}, model_path)
print(f"PyTorch model saved to: {model_path}")

# Save imputer for preprocessing
imputer_path = model_dir + "/imputer.pkl"
joblib.dump(imputer, imputer_path)
print(f"Imputer saved to: {imputer_path}")

# Save feature column names for inference
feature_names_path = model_dir + "/feature_names.pkl"
joblib.dump(list(X_train.columns), feature_names_path)
print(f"Feature names saved to: {feature_names_path}")

# Save visualizations
cm_fig.savefig(images_dir + "/confusion_matrix.png", dpi=100, bbox_inches='tight')
print(f"Confusion matrix saved to: {images_dir}/confusion_matrix.png")

training_history_fig.savefig(images_dir + "/training_history.png", dpi=100, bbox_inches='tight')
print(f"Training history saved to: {images_dir}/training_history.png")

pr_curve_fig.savefig(images_dir + "/pr_curve.png", dpi=100, bbox_inches='tight')
print(f"PR curve saved to: {images_dir}/pr_curve.png")

print("\n" + "=" * 80)
print("ALL ARTIFACTS SAVED SUCCESSFULLY")
print("=" * 80)

## Create Predictor Script

In [ ]:
predictor_code = '''
import os
import joblib
import torch
import torch.nn as nn
import numpy as np

class FraudDetectorNN(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.3):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 15000),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(15000, 10000),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(5000, 1)
        )
    
    def forward(self, x):
        return self.network(x)

class Predict:
    def __init__(self):
        self.model = None
        self.imputer = None
        self.feature_names = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    def load_model(self, model_dir):
        # Load model checkpoint
        checkpoint = torch.load(os.path.join(model_dir, "cc_fraud_nn_model.pt"), 
                               map_location=self.device)
        
        # Initialize and load model
        self.model = FraudDetectorNN(
            input_dim=checkpoint['input_dim'],
            dropout_rate=checkpoint['dropout_rate']
        ).to(self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        # Load imputer
        self.imputer = joblib.load(os.path.join(model_dir, "imputer.pkl"))
        
        # Load feature names
        self.feature_names = joblib.load(os.path.join(model_dir, "feature_names.pkl"))
    
    def predict(self, inputs):
        import pandas as pd
        
        # Convert to DataFrame if needed
        if isinstance(inputs, dict):
            inputs = pd.DataFrame([inputs])
        elif isinstance(inputs, list):
            inputs = pd.DataFrame(inputs)
        
        # Ensure correct column order
        inputs = inputs[self.feature_names]
        
        # Preprocess
        X = self.imputer.transform(inputs)
        X_tensor = torch.FloatTensor(X).to(self.device)
        
        # Predict
        with torch.no_grad():
            probs = torch.sigmoid(self.model(X_tensor)).cpu().numpy().squeeze()
        
        # Return predictions
        if probs.ndim == 0:
            probs = np.array([probs])
        
        return {"predictions": (probs >= 0.5).astype(int).tolist(),
                "probabilities": probs.tolist()}
'''

predictor_script = "ccfraud-nn-predictor.py"
with open(os.path.join(model_dir, predictor_script), 'w') as f:
    f.write(predictor_code.strip())

print(f"Predictor script saved to: {model_dir}/{predictor_script}")

## Register Model in Hopsworks

In [ ]:
print("=" * 80)
print("REGISTERING MODEL IN HOPSWORKS")
print("=" * 80)

# Format metrics for model registry
metrics_for_registry = {
    'pr_auc': f"{metrics_dict['pr_auc']:.4f}",
    'precision': f"{metrics_dict['precision']:.4f}",
    'recall': f"{metrics_dict['recall']:.4f}",
    'f1_score': f"{metrics_dict['f1_score']:.4f}",
    'accuracy': f"{metrics_dict['accuracy']:.4f}",
    'class_weight_ratio': f"{class_weight_ratio:.2f}",
    'n_train_samples': str(len(y_train)),
    'n_fraud_train': str(n_positive),
    'imbalance_ratio': f"{class_weight_ratio:.2f}",
    'model_type': 'neural_network',
    'architecture': '15000-10000-5000-1',
    'dropout_rate': '0.3',
    'weight_decay': '0.01'
}

print("Model metadata:")
for key, value in metrics_for_registry.items():
    print(f"  {key:20s}: {value}")

model_name = "cc_fraud_nn_model"

# Create model in registry
cc_fraud_nn_model = mr.python.create_model(
    name=model_name,
    metrics=metrics_for_registry,
    feature_view=fv,
    description="Credit Card Fraud Detection - PyTorch Neural Network. "
                "3-layer feedforward network (15k-10k-5k neurons) with dropout and L2 regularization. "
                f"Trained on {len(y_train):,} samples with {n_positive} fraud cases. "
                f"Uses {input_dim} features after MinMaxScaler and OneHotEncoder transformations."
)

# Upload model directory to registry
cc_fraud_nn_model.save(model_dir)

print("\n" + "=" * 80)
print("MODEL REGISTRATION COMPLETE")
print("=" * 80)
print(f"Model name: {model_name}")
print(f"Version: {cc_fraud_nn_model.version}")
print(f"Feature View: cc_fraud_fv v2 (with MinMaxScaler + OneHotEncoder)")

## Optional: Deploy Model

In [ ]:
Uncomment to deploy the model

ms = proj.get_model_serving()
best_model = mr.get_best_model(name=model_name, metric="f1_score", direction="max")

if best_model.version == cc_fraud_nn_model.version:
    print(f"This is the best model version at: {best_model.version_path}")
    predictor_path = os.path.join(best_model.version_path, f"Files/{predictor_script}")
    deployment_name = "ccfraud-nn"
    try:
        deployment = ms.get_deployment(deployment_name)
        deployment.delete(force=True)
        print(f"Deleted deployment {deployment_name}")
    except:
        print("Deployment not running")
    deployment = best_model.deploy(
        name=deployment_name, 
        script_file=predictor_path, 
        environment="pytorch-inference-pipeline"
    )
    deployment.start(await_running=180)
    deployment_state = deployment.get_state().describe()
else:
    print("Not deploying this model, as its performance is worse than the existing deployment")

In [ ]:
print("=" * 80)
print("NOTEBOOK COMPLETE")
print("=" * 80)
print(f"\nSummary:")
print(f"  - Feature View: cc_fraud_fv v2 (with transformations)")
print(f"  - Model: 3-layer Neural Network ({total_params:,} parameters)")
print(f"  - Architecture: 15,000 -> 10,000 -> 5,000 -> 1 neurons")
print(f"  - Regularization: L2 (weight_decay=0.01) + Dropout (0.3)")
print(f"  - Class imbalance: pos_weight={class_weight_ratio:.2f}")
print(f"\nMetrics:")
print(f"  - PR-AUC: {pr_auc:.4f}")
print(f"  - Precision: {precision:.4f}")
print(f"  - Recall: {recall:.4f}")
print(f"  - F1-Score: {f1:.4f}")